In [1]:
import pandas as pd
import os
from tqdm.auto import tqdm

In [2]:
projecs = ['TCGA-ACC', 'TCGA-BLCA', 'TCGA-BRCA', 'TCGA-CESC', 'TCGA-CHOL', 'TCGA-COAD', 'TCGA-DLBC',
           'TCGA-ESCA', 'TCGA-HNSC', 'TCGA-KICH', 'TCGA-KIRC', 'TCGA-KIRP',
           'TCGA-LGG', 'TCGA-LIHC', 'TCGA-LUAD', 'TCGA-LUSC', 'TCGA-MESO', 'TCGA-OV', 'TCGA-PAAD',
           'TCGA-PCPG', 'TCGA-PRAD', 'TCGA-READ', 'TCGA-SARC', 'TCGA-SKCM', 'TCGA-STAD', 'TCGA-TGCT',
           'TCGA-THCA', 'TCGA-THYM', 'TCGA-UCEC', 'TCGA-UCS', 'TCGA-UVM']

In [ ]:
pairs = pd.read_csv('/home/Mount_sda1/alexmakh/data_to_paper/ts_res_full.tsv', sep='\t')
pairs = pairs.sort_values('CWCS')[['isomiRNA', 'Gene Symbol', 'CWCS']].copy()

pairs = pairs.rename(columns={'isomiRNA': 'isomiR'})
pairs.to_csv('/home/Mount_sda1/alexmakh/isomiRNA/interactions.tsv', sep='\t', index=None)

In [ ]:
#interaction files
alex_path = '/home/alexgorbonos/huge/alexgorbonos/isomiR_old/TCGA_data/'
files = os.listdir(alex_path)
len(files)

In [ ]:
for filename in tqdm(files):
    data = pd.read_csv(alex_path + filename, sep='\t', index_col=0)
    filename = filename[:-8]
    molecules = set(data.index)
    mirdb = pd.read_csv(f'/home/alexgorbonos/huge/alexgorbonos/isomiR_research/miRDB/predicted_targets/{filename}.tsv', sep='\t').groupby('isomiR').size().to_dict()
    
    pairs_tmp = pairs.copy()
    pairs_tmp['isomiR_true_false'] = pairs_tmp['isomiR'].apply(lambda x: x in molecules)
    pairs_tmp['gene_true_false'] = pairs_tmp['Gene Symbol'].apply(lambda x: x in molecules)
    pairs_tmp = pairs_tmp[pairs_tmp['isomiR_true_false'] & pairs_tmp['gene_true_false']]
    pairs_tmp = pairs_tmp[['isomiR', 'Gene Symbol', 'CWCS']]
    new_pairs = pd.DataFrame({}, columns=['isomiR', 'Gene Symbol'])
    for mir, count in tqdm(mirdb.items()):
        tmp = pairs_tmp[pairs_tmp['isomiR'] == mir].sort_values('CWCS').drop('CWCS', axis=1).iloc[:count]
        new_pairs = pd.concat([new_pairs, tmp])
    
    print(f'{filename} interactions count: {new_pairs.shape[0]}')
    new_pairs.to_csv(f'/home/alexmakh/huge/alexmakh/isomiRNA/TCGA_data/{filename}.tsv', sep='\t', index=None)